In [13]:
from scipy.stats import norm
from scipy.stats import t
import numpy as np
import math

S0 = 100
r = 0.05
q = 0.02
T = 0.4
sigma = 0.5
K1 = 102
K2 = 1000000000000000000
K3 = 10000000000000000000
K4 = 100000000000000000000000



def black_scholes(S, r, q, T, K1, K2, K3, K4, sigma):
    d11 = (math.log(S0 / K1) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d12 = (math.log(S0 / K2) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d13 = (math.log(S0 / K3) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d14 = (math.log(S0 / K4) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))

    d21 = (math.log(S0 / K1) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d22 = (math.log(S0 / K2) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d23 = (math.log(S0 / K3) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d24 = (math.log(S0 / K4) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))

    call = (S0 * math.exp(-q*T) * ((norm.cdf(d11)-norm.cdf(d12)) + ((K2-K1)/(K3-K4)) * (norm.cdf(d13)-norm.cdf(d14)))
    + math.exp(-r*T) * (-K1*(norm.cdf(d21)-norm.cdf(d23))+K2*(norm.cdf(d22)-norm.cdf(d23))-((K2-K1)/(K3-K4))*K4*(norm.cdf(d23)-norm.cdf(d24))))

    return call


call_price = black_scholes(S0,r,q,T,K1,K2,K3,K4,sigma)

print(f"Call price: {call_price:.4f}")

Call price: 12.1272


Monte-Carlo's simulation

In [15]:
import numpy as np

# 初始化參數
n_simulation = 10000
n_steps = 20

call_prices = []
for _ in range(n_steps):
    rn = np.random.normal(0, 1, n_simulation)
    ST = S0 * np.exp((r - q - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * rn)
    payoff = np.where(ST < K2, np.exp(-r*T) * np.maximum(ST - K1, 0),
                      np.where(ST < K3, np.exp(-r*T) * (K2 - K1),
                               np.where(K3< ST, np.exp(-r*T) * np.maximum(((K2 - K1) / (K3 - K4)) * (ST - K4), 0),0)))
    average = np.mean(payoff)
    call_prices.append(average)

mean = np.mean(call_prices)
std = np.std(call_prices)
upper = mean + 2 * std
lower = mean - 2 * std

print(f"mean price: {mean:.4f}", f"std of price: {std:.4f}", f"lower bound: {lower:.4f}", f"upper bound: {upper:.4f}")

mean price: 12.2387 std of price: 0.1753 lower bound: 11.8880 upper bound: 12.5894
